In [3]:
# Load point clouds from file or generate them
import open3d as o3d
import numpy as np

from RGBD2PointCloud import RGBD2PtsCloud
from tqdm import tqdm
rgb2ptcs = RGBD2PtsCloud(0)

In [12]:
sequence_length = 2176

trans_est = np.zeros((sequence_length-1, 3))
trans_est_norm = np.zeros(sequence_length-1)

# for idx in range(sequence_length-1):
for idx in tqdm(range(sequence_length-1)):
    rgb2ptcs.frame = idx
    pcd_dense0, _ = rgb2ptcs.convert6()
    source = pcd_dense0


    rgb2ptcs.frame = idx+1
    pcd_dense1, _ = rgb2ptcs.convert6()
    target = pcd_dense1

    target.estimate_normals()

    current_transformation = np.identity(4)

    result_icp = o3d.pipelines.registration.registration_icp(
    source, target, 0.02, current_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

    # print(result_icp)
    # print(result_icp.transformation[:3,3])
    # print(np.linalg.norm(result_icp.transformation[:3,3]))

    # print("Result is:", result_icp)

    trans_est[idx] = result_icp.transformation[:3,3]
    trans_est_norm[idx] = np.linalg.norm(result_icp.transformation[:3,3])



In [ ]:
# save the trans_est and trans_est_norm to a file
np.save('trans_est.npy',trans_est)
np.save('trans_est_norm.npy',trans_est_norm)

In [11]:
data_path = '/home/data2/TartanAir/TartanAir_comb/abandonedfactory/Easy/P000/pose_left.txt'
data_pose = np.loadtxt(data_path)
data_pose.shape


array([0.14437977, 0.13975827])

In [ ]:
import pypose as pp
gt_trans = np.zeros((sequence_length-1, 3))
gt_trans_norm = np.zeros(sequence_length-1)
for idx in tqdm(range(sequence_length-1)):
    pose0 = pp.SE3(data_pose[idx])
    print(pose0)
    pose1 = pp.SE3(data_pose[idx+1])
    print(pose1)
    diff = pose0.Inv() * pose1
    print(diff.translation())
    print(diff.translation().norm()) 

    gt_trans[idx] = diff.translation()
    gt_trans_norm[idx] = diff.translation().norm()

In [6]:
# examples/Python/Advanced/colored_pointcloud_registration.py

import numpy as np
import copy
import open3d as o3d

def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])



import matplotlib.pyplot as plt
if __name__ == "__main__":

    print("1. Load two point clouds and show initial pose")
    # source = o3d.io.read_point_cloud("../../TestData/ColoredICP/frag_115.ply")
    # target = o3d.io.read_point_cloud("../../TestData/ColoredICP/frag_116.ply")

    # draw initial alignment
    current_transformation = np.identity(4)
    # draw_registration_result_original_color(source, target,
    #                                         current_transformation)
    # plt.show()
    
    # point to plane ICP
    current_transformation = np.identity(4)
    print("2. Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. Distance threshold 0.02.")
    target.estimate_normals()
    result_icp = o3d.pipelines.registration.registration_icp(
        source, target, 0.02, current_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    print(result_icp)
    # draw_registration_result_original_color(source, target,
    #                                         result_icp.transformation)

    print(result_icp.transformation[:3,3])
    # calculate norm of translation

    print(np.linalg.norm(result_icp.transformation[:3,3]))

    print("Result is:", result_icp)

1. Load two point clouds and show initial pose
2. Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.02.
RegistrationResult with fitness=9.656683e-01, inlier_rmse=3.939879e-03, and correspondence_set size of 200859
Access transformation to get result.
[ 0.13182905  0.03436155 -0.04781112]
0.14437976555083598
Result is: RegistrationResult with fitness=9.656683e-01, inlier_rmse=3.939879e-03, and correspondence_set size of 200859
Access transformation to get result.


In [15]:
import numpy as np
import pypose as pp
data = [
    [-1.179049015045166016e+01, -4.795211315155029297e+00, -1.666950345039367676e+00, -0.000000000000000000e+00, -0.000000000000000000e+00, 7.702344059944152832e-01, 6.377608776092529297e-01],
    [-1.165132999420166016e+01, -4.815492630004882812e+00, -1.634087443351745605e+00, 1.423883717507123947e-02, -1.018611807376146317e-02, 7.771025300025939941e-01, 6.291303634643554688e-01]
]

array_data = np.array(data)
# print(array_data)

pose0 = pp.SE3(array_data[0])
print(pose0)
pose1 = pp.SE3(array_data[1])
print(pose1)
diff = pose0.Inv() * pose1
print(diff.translation())
print(diff.translation().norm()) 

SE3Type LieTensor:
LieTensor([-11.7905,  -4.7952,  -1.6670,  -0.0000,  -0.0000,   0.7702,   0.6378])
SE3Type LieTensor:
LieTensor([-1.1651e+01, -4.8155e+00, -1.6341e+00,  1.4239e-02, -1.0186e-02,
            7.7710e-01,  6.2913e-01])
tensor([-0.0459, -0.1329,  0.0329])
tensor(0.1444)


In [17]:
trans_err = diff.translation().norm() - np.linalg.norm(result_icp.transformation[:3,3])
print(trans_err)

tensor(3.6836e-05)


In [20]:
data_path = '/home/data2/TartanAir/TartanAir_comb/abandonedfactory/Easy/P000/pose_left.txt'
data_pose = np.loadtxt(data_path)
data_pose.shape


(2176, 7)